In [3]:
import openai
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import os

print(os.getcwd())
# change the directory to the root of the project first to parent
os.chdir("..")
os.chdir("Data")



c:\Users\megar\OneDrive\Documenten\GitHub\CustomerAnalytics\Data


In [4]:
# import the data from csv
unique_products = pd.read_csv("unique_products.csv")

# add 
unique_products['Brand'] = unique_products['Product_name'].apply(lambda x: "Knorr" if "Knorr" in x else "Heilmans")

# for the first analysis I only want Knorr products
unique_products_knorr = unique_products[unique_products['Brand'] == "Knorr"]

# For the analysis I also need a list of the categories on the website
categories = unique_products_knorr['Closest_Category'].unique()


In [9]:
# Your OpenAI API key
openai.api_key = '#'  # Replace with your actual API key

import openai

# Ensure you have set your OpenAI API key somewhere in your code:
# openai.api_key = 'your_api_key_here'

def classify_product(product, categories=categories):
    """
    Classifies the product based on the categories from the Knorr website.
    """
    # Ensure 'categories' is either passed as a parameter or defined within the function
    categories_str = ', '.join(categories)
    system_message = f"You are a helpful assistant for categories based on product names. You can choose between the following categories: {categories_str}. Classify the product based on the product title, output only the name of the category."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": product}
        ]
    )

    answer = response.choices[0].message['content'].strip()
    print(answer)
    return answer

# # get me 5 product to test the model
# products = unique_products_knorr['Product_name'].sample(5)

# #test function
# for product in products:
#     print(f"Product: {product}")
#     print(f"Category: {classify_product(product, categories)}")
#     print(f"Category closest to the product: {unique_products_knorr[unique_products_knorr['Product_name'] == product]['Closest_Category'].values[0]}")
#     print()

def classify_product_parallel(df, num_threads=10):
    results = [None] * len(df)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_index = {executor.submit(classify_product, product): i for i, product in enumerate(df['Product_name'])}
        for future in as_completed(future_to_index):
            index = future_to_index[future]
            try:
                results[index] = future.result()
            except Exception as e:
                print(f"Error in processing product at index {index}: {e}")

    return results

# Load your DataFrame here
# Example: test = pd.read_csv('path_to_your_file.csv')

# Start timer
start_time = time.time()

# Classify tweets in parallel
unique_products_knorr['GPT_estimated'] = classify_product_parallel(unique_products_knorr)

# End timer
end_time = time.time()

# Print execution time
print("Time taken for classification:", end_time - start_time)


Fix Produkte | Knorr Deutschland
Sauces
Suppen
Grillsaucen
Die Bio Gemüse Bouillon
Fix Produkte | Knorr Deutschland
Fix Produkte | Knorr Deutschland
Würzmittel | Knorr Deutschland
Fix Produkte | Knorr Deutschland
Würzmittel | Knorr Deutschland
Fix Produkte | Knorr Deutschland
Fix Produkte | Knorr Deutschland
Die Bio Gemüse Bouillon
Sauces
Salatzutaten
Grillsaucen
Ketchup & Mayo
Grillsaucen
Snacks & Fertiggerichte
Fix Produkte | Knorr Deutschland
Grillsaucen
Fix Produkte | Knorr Deutschland
Sauces
Fix Produkte | Knorr Deutschland
Salatzutaten
Brühen, Bouillons & Würzmittel
Fix Produkte | Knorr Deutschland
Brühen, Bouillons & Würzmittel
Grillsaucen
Brühen, Bouillons & Würzmittel
Fix Produkte | Knorr Deutschland
Fix Produkte | Knorr Deutschland
Fix Produkte | Knorr Deutschland
Fix Produkte | Knorr Deutschland
Salatzutaten
Suppen
Snacks & Fertiggerichte
Die Bio Gemüse Bouillon
Suppen
Fix Produkte | Knorr Deutschland
Suppen
Sauces
Suppen
Fix Produkte | Knorr Deutschland
Suppen
Suppen
Fix Pr

C:\Users\megar\AppData\Local\Temp\ipykernel_33952\19701609.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_products_knorr['GPT_estimated'] = classify_product_parallel(unique_products_knorr)


NameError: name 'test' is not defined

In [11]:
# lets look at how different GPT and closest product is:

# how many are the same between closest_category and GPT_estimate
print(f"Same: {unique_products_knorr[unique_products_knorr['Closest_Category'] == unique_products_knorr['GPT_estimated']].shape[0]}")

#make a visualisation with the closest_category and GPT_estimate difference


Same: 191


In [12]:
# Save the dataframe
write_path = "unique_products_knorr_GPT.csv"
unique_products_knorr.to_csv(write_path, index=False)